In [1]:
import sys  
sys.path.insert(1, '/Users/asifahmed/Documents/Codes/MyModelProject')


from data_handling.bank_data import BankData
from models.model_trainer import ModelTrainer
from evaluation.evaluator import Evaluator
from data_handling.adult_data import Adult
from visualization.visualizer import Visualizer
from data_handling.givemecredit_data import GiveMeCredit
from data_handling.crdit_data import Credit
from explainability.explainer_factory import get_explainer
import pandas as pd

In [2]:
credit_instance = Credit()
credit_instance.load_csv('/Users/asifahmed/Documents/Codes/MyModelProject/datasets/processed/credit_processed.csv')
credit_instance.preview_data()
credit_instance.sample_data(5000)
# credit_instance.print_summary_statistics()
# credit_instance.print_features_names()
# credit_instance.plot_histogram()

CSV file loaded successfully with delimiter: ','
Data has been sampled. New data size: 5000 rows.


In [3]:
trainer = ModelTrainer()
# Split the data
trainer.split_data(credit_instance.data, target_column='NoDefaultNextMonth')
model = trainer.train('svm')
evaluator = Evaluator(model, trainer.X_test, trainer.y_test)
evaluator.report()

Data split into train and test sets.
svm model trained successfully.
Accuracy: 0.774
Precision: 0.7973273942093542
Recall: 0.9421052631578948
F1 Score: 0.8636911942098915
Confusion Matrix:
 [[ 58 182]
 [ 44 716]]
Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.24      0.34       240
           1       0.80      0.94      0.86       760

    accuracy                           0.77      1000
   macro avg       0.68      0.59      0.60      1000
weighted avg       0.74      0.77      0.74      1000



In [12]:
continuous_features = ['Age', 'EducationLevel', 'MaxPaymentAmountOverLast6Months', 'MonthsWithZeroBalanceOverLast6Months', 'MonthsWithLowSpendingOverLast6Months', 'MonthsWithHighSpendingOverLast6Months', 'MostRecentBillAmount', 'MostRecentPaymentAmount', 'TotalOverdueCounts', 'TotalMonthsOverdue', 'HistoryOfOverduePayments']
explainer = get_explainer('dice', model=model, data=credit_instance.data,
                          continuous_features=continuous_features,
                          outcome_name='NoDefaultNextMonth')
y_pred = model.predict(trainer.X_test)

# Step 2: Convert predictions to a Series for easy boolean indexing
y_pred_series = pd.Series(y_pred, index=trainer.X_test.index)

# Step 3: Filter the instances where predictions are 1
predicted_defaults = trainer.X_test[y_pred_series == 0]
# Now, select the first few instances for explanation. Adjust the number as needed.
query_instance = predicted_defaults.iloc[0:2]
counterfactuals = explainer.generate_explanation(query_instance, total_CFs=1)


100%|██████████| 2/2 [00:00<00:00, 25.90it/s]

Query instance (original outcome : 0)


,Married,Age,Sex,EducationLevel,MaxBillAmountOverLast6Months,MaxPaymentAmountOverLast6Months,MonthsWithZeroBalanceOverLast6Months,MonthsWithLowSpendingOverLast6Months,MonthsWithHighSpendingOverLast6Months,MostRecentBillAmount,MostRecentPaymentAmount,TotalOverdueCounts,TotalMonthsOverdue,HistoryOfOverduePayments,NoDefaultNextMonth
0,0,35,2,2,1530,120,0,0,0,1310,60,1,2,1,0



Diverse Counterfactual set (new outcome: 1)


,Married,Age,Sex,EducationLevel,MaxBillAmountOverLast6Months,MaxPaymentAmountOverLast6Months,MonthsWithZeroBalanceOverLast6Months,MonthsWithLowSpendingOverLast6Months,MonthsWithHighSpendingOverLast6Months,MostRecentBillAmount,MostRecentPaymentAmount,TotalOverdueCounts,TotalMonthsOverdue,HistoryOfOverduePayments,NoDefaultNextMonth
0,-,-,-,-,-,-,-,-,-,-,6211,-,-,-,1


Query instance (original outcome : 0)


,Married,Age,Sex,EducationLevel,MaxBillAmountOverLast6Months,MaxPaymentAmountOverLast6Months,MonthsWithZeroBalanceOverLast6Months,MonthsWithLowSpendingOverLast6Months,MonthsWithHighSpendingOverLast6Months,MostRecentBillAmount,MostRecentPaymentAmount,TotalOverdueCounts,TotalMonthsOverdue,HistoryOfOverduePayments,NoDefaultNextMonth
0,0,22,1,2,360,60,0,0,0,270,30,1,2,1,0



Diverse Counterfactual set (new outcome: 1)


,Married,Age,Sex,EducationLevel,MaxBillAmountOverLast6Months,MaxPaymentAmountOverLast6Months,MonthsWithZeroBalanceOverLast6Months,MonthsWithLowSpendingOverLast6Months,MonthsWithHighSpendingOverLast6Months,MostRecentBillAmount,MostRecentPaymentAmount,TotalOverdueCounts,TotalMonthsOverdue,HistoryOfOverduePayments,NoDefaultNextMonth
0,-,-,-,-,-,-,-,-,-,-,12850,-,-,-,1
